In [1]:
# importing libraries
import numpy as np
import pandas as pd
from pathlib import Path
import hvplot.pandas
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import time
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import MeanSquaredError
from pandas.tseries.offsets import DateOffset
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from bokeh.models.formatters import NumeralTickFormatter
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


2023-08-01 21:19:32.647031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
all_values = pd.read_csv(Path("Resources/all_values.csv"))
all_values['date'] = pd.to_datetime(all_values['date'], format='%Y-%m')
all_values.set_index('date', inplace=True)

In [3]:
scaler_all = StandardScaler()
all_scaler = scaler_all.fit(all_values)
all_values_scaled = all_scaler.transform(all_values)

all_values_scaled_df = pd.DataFrame(all_values_scaled, columns=all_values.columns, index=all_values.index)
all_values_scaled_df.head()

,Composite,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2008-06-01,-1.131020,-0.672479,-0.469992,-0.992346,-0.858730,2.172972,-1.310833,-1.693840,-1.168618,-1.911253,0.170914,-0.692024,-1.836365,-1.370649,-1.488305,-1.180402,-1.390596
2008-07-01,-1.137164,-0.645519,-0.552576,-1.105315,-0.943586,2.172972,-1.264221,-1.641446,-1.088744,-1.884173,0.677639,-0.692024,-1.823707,-1.386239,-1.493945,-1.150188,-1.380045
2008-08-01,-1.142285,-0.612804,-0.453057,-0.986607,-0.873159,2.172972,-1.264221,-1.589052,-1.048807,-1.884173,0.360936,-0.803609,-1.709782,-1.339471,-1.499586,-1.170331,-1.358941
2008-09-01,-1.146893,-0.728685,-0.496322,-0.872477,-0.765473,2.172972,-1.254898,-1.510461,-1.104719,-1.762314,0.170914,-0.810583,-1.722441,-1.355060,-1.471383,-1.170331,-1.358941
2008-10-01,-1.163279,-0.891076,-1.336555,-1.199396,-1.016262,1.332881,-1.329479,-1.477714,-1.096731,-1.789394,-0.652515,-1.089543,-1.709782,-1.386239,-1.471383,-1.281118,-1.380045


In [4]:
y = all_values_scaled_df["Composite"]

In [5]:
X = all_values_scaled_df.drop(columns = "Composite")

In [74]:
y

date
2008-06-01   -1.131020
2008-07-01   -1.137164
2008-08-01   -1.142285
2008-09-01   -1.146893
2008-10-01   -1.163279
                ...   
2022-12-01    1.712419
2023-01-01    1.635611
2023-02-01    1.604375
2023-03-01    1.631514
2023-04-01    1.728292
Name: Composite, Length: 179, dtype: float64

In [76]:
all_values_scaled_df.hvplot(
    xlabel='Date', 
    ylabel='Scaled Value', 
    title='Scaled Values',
    height=800,
    width=2000
)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [77]:
#training_begin = X.index.min()
#print(training_begin)
#training_end = X.index.min() + DateOffset(months=125)
#print(training_end)

In [78]:
#X_train = X.loc[training_begin:training_end]
#y_train = y.loc[training_begin:training_end]
#X_train.shape

In [79]:
#X_test = X.loc[training_end:]
#y_test = y.loc[training_end:]
#X_test.shape

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
#cheking for feature importance
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
model = RandomForestRegressor(n_estimators=100, random_state=40)
model.fit(X_train,y_train)

RandomForestRegressor(random_state=40)

In [9]:
feature_names = all_values_scaled_df.columns

In [10]:
feature_names = all_values.columns.tolist()


In [11]:
rf = RandomForestRegressor()
rf.fit(X, y)
results = permutation_importance(rf, X, y)

In [12]:
importance_scores = results.importances_mean

In [13]:
for feature_name, importance_score in zip(feature_names, importance_scores):
    print(f"{feature_name}: {importance_score}")

Composite: 0.004921327653259433
Lumber: 0.0027076135360931453
Wood: 0.0004472488350809378
XHB: 0.0021496063757701967
ITB: 0.000368808087049044
Bank Rate: 0.0299819989562077
All-items 8: 0.004310181701017113
Food: 0.04742261346778591
Shelter: 0.0025590168855728957
Household operations, furnishings and equipment: 0.000581491725136818
Clothing and footwear: 0.0048829749852780545
Transportation: 0.035026352786304685
Health and personal care: 0.026597606615310875
Recreation, education and reading: 0.01687422872178044
Alcoholic beverages, tobacco products and recreational cannabis: 0.03065798504756423
All-items excluding food: 0.03342369912492449


In [14]:
# Sort the features based on their importance scores in descending order
sorted_indices = importance_scores.argsort()[::-1]



In [15]:
# Select the top 4 features
top_4_features = sorted_indices[:4]



In [16]:
top_4_features

array([ 7, 11, 15, 14])

In [17]:
sorted_indices

array([ 7, 11, 15, 14,  5, 12, 13,  0, 10,  6,  1,  8,  3,  9,  2,  4])

In [19]:
# Create a new dataset containing only the top 4 features
X_reduced = X.iloc[:, top_4_features]

In [20]:
# Retrain the model using the reduced dataset
model_reduced = RandomForestRegressor()
model_reduced.fit(X_reduced, y)

# Make predictions on the test set using the reduced model
y_pred_reduced = model_reduced.predict(X_test.iloc[:, top_4_features])

# Evaluate the reduced model's performance using MSE
mse_reduced = mean_squared_error(y_test, y_pred_reduced)
print(f"Mean Squared Error (Reduced Model): {mse_reduced}")

Mean Squared Error (Reduced Model): 0.0003218343159549394


In [81]:
# Neural network parameters
n_input_feats = len(X.columns)

n_outputs = 1

hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32
# hidden_nodes_layer4 = 4
print(f"""
Neural Network Parameters:
--------------------------
Input Features: {n_input_feats}
Layers:         3
Layer1 Nodes:   {hidden_nodes_layer1}
Layer2 Nodes:   {hidden_nodes_layer2}
Layer3 Nodes:   {hidden_nodes_layer3}
Output Nodes:   {n_outputs}
""")


Neural Network Parameters:
--------------------------
Input Features: 16
Layers:         3
Layer1 Nodes:   128
Layer2 Nodes:   64
Layer3 Nodes:   32
Output Nodes:   1



In [82]:
# define model
nn = Sequential()
# Add the first hidden layer
nn.add(Dense(
    units=hidden_nodes_layer1,
    input_dim = n_input_feats,
    activation='relu',
    name='hidden1'
))
# Add the second hidden layer
nn.add(Dense(
    units=hidden_nodes_layer2,
    activation='relu',
    name='hidden2'
))
# Add the third hidden layer
nn.add(Dense(
    units=hidden_nodes_layer3,
    activation='relu',
    name='hidden3'
))
# Add output
nn.add(Dense(
    n_outputs,
    activation='linear',
    name='output'
))

In [83]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 128)               2176      
                                                                 
 hidden2 (Dense)             (None, 64)                8256      
                                                                 
 hidden3 (Dense)             (None, 32)                2080      
                                                                 
 output (Dense)              (None, 1)                 33        
                                                                 
Total params: 12,545
Trainable params: 12,545
Non-trainable params: 0
_________________________________________________________________


In [84]:
# Compile the Sequential model
nn.compile(loss='mean_squared_error', optimizer='adam', metrics=[MeanSquaredError()])

In [85]:
# Fit the model 
model = nn.fit(
    X_train,
    y_train,
    batch_size=10,
    validation_split=0.2,
    epochs=100
)

Epoch 1/100
11/11 [==============================] - 1s 23ms/step - loss: 0.8283 - mean_squared_error: 0.8283 - val_loss: 0.0617 - val_mean_squared_error: 0.0617
Epoch 2/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0762 - mean_squared_error: 0.0762 - val_loss: 0.1021 - val_mean_squared_error: 0.1021
Epoch 3/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0718 - mean_squared_error: 0.0718 - val_loss: 0.0190 - val_mean_squared_error: 0.0190
Epoch 4/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0381 - mean_squared_error: 0.0381 - val_loss: 0.0122 - val_mean_squared_error: 0.0122
Epoch 5/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0229 - mean_squared_error: 0.0229 - val_loss: 0.0139 - val_mean_squared_error: 0.0139
Epoch 6/100
11/11 [==============================] - 0s 5ms/step - loss: 0.0181 - mean_squared_error: 0.0181 - val_loss: 0.0073 - val_mean_squared_error: 0.0073
Epoch 7/100
11/11 [==============

In [86]:
# R2 score for training
train_pred = nn.predict(X_train)
training_r2 = r2_score(y_train.values, train_pred)
training_r2

5/5 [==============================] - 0s 2ms/step


0.9991402013730172

In [87]:
train_predictions_df = X_train.copy()
train_predictions_df['Train Predictions'] = train_pred
cols = train_predictions_df.columns.tolist()
cols = [cols[-1]] + cols[:-1]
train_predictions_df = train_predictions_df[cols]

In [88]:
train_pred_descaled_df = pd.DataFrame(all_scaler.inverse_transform(train_predictions_df), columns=train_predictions_df.columns, index=train_predictions_df.index)
#train_pred_descaled_df['Actual'] = all_values['Composite'][:126].values
train_pred_descaled_df

,Train Predictions,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2021-03-01,785230.031230,1009.10,85.302173,66.461305,64.253044,0.50,139.3,155.0,150.0,123.4,93.6,147.9,130.9,115.8,173.5,136.4,133.3
2012-02-01,382057.565232,275.40,40.709000,19.962500,13.996000,1.25,121.4,129.8,126.9,112.6,93.0,128.4,118.5,105.1,136.6,119.7,115.7
2022-11-01,899032.858661,460.00,74.312381,59.850952,57.723333,4.00,154.3,177.8,167.2,132.8,95.2,170.1,141.2,124.8,184.7,150.0,144.3
2011-03-01,357597.919528,323.80,48.085653,17.892609,13.270435,1.25,119.1,126.8,124.6,110.3,91.6,124.2,116.7,105.6,134.8,117.6,114.2
2009-12-01,336402.491721,241.00,39.199091,14.875909,11.698182,0.50,115.4,122.1,121.3,107.8,93.5,117.0,113.4,103.7,131.2,114.1,111.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-01,630926.935688,394.10,58.924091,41.976364,38.715909,2.00,136.4,150.8,144.3,123.4,95.7,142.3,127.9,119.2,170.8,133.6,130.8
2019-11-01,632810.766000,421.30,65.139500,45.670500,44.909500,2.00,136.8,151.7,145.9,123.6,97.1,142.8,128.0,115.8,172.1,133.9,131.1
2014-06-01,429256.933310,339.54,52.693333,32.176190,24.291905,1.25,125.4,135.9,132.2,116.1,93.7,131.3,118.8,107.4,146.5,123.3,118.9


In [89]:
# Create a DataFrame with the history dictionary
model_df = pd.DataFrame(model.history, index=range(1, len(model.history["loss"]) + 1))

In [90]:
model_df[['val_loss', 'mean_squared_error']].hvplot(
height=800,
    width=2000
)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [91]:
predictions = nn.predict(X_test)

2/2 [==============================] - 0s 2ms/step


In [92]:
# R2 score from test
test_r2 = r2_score(y_test.values, predictions)
test_r2

0.9973623638831222

In [93]:
predictions_df = X_test.copy()
predictions_df['Test Predictions'] = predictions
cols = predictions_df.columns.tolist()
# Move the last column to the first position
cols = [cols[-1]] + cols[:-1]
# Reindex the DataFrame with the new column order
predictions_df = predictions_df[cols]
predictions_df.head()

,Test Predictions,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2017-12-01,0.306142,0.098641,0.866361,0.457344,0.614173,-0.067270,0.264677,0.179244,0.181256,0.404081,0.487617,0.207623,0.277562,0.500081,0.508433,0.290051,0.318794
2015-07-01,-0.341248,-0.521755,-0.226566,0.059789,-0.214154,-0.627330,-0.173483,-0.049979,-0.250064,0.187442,0.614298,-0.364246,-0.266742,-0.232622,-0.112022,-0.213529,-0.219348
2016-07-01,0.019796,-0.352269,-0.510179,-0.033368,-0.141799,-0.627330,-0.042967,0.087555,-0.074341,0.512401,0.297595,-0.399116,-0.089527,0.063577,0.158722,-0.072527,0.033895
2021-10-01,1.871723,1.365447,1.812700,2.226979,2.044376,-0.907361,1.392705,1.266419,1.475219,1.053999,0.170914,1.518737,1.391488,1.341909,1.252979,1.428141,1.384525
2011-05-01,-0.907956,-0.638818,-0.473829,-0.981466,-1.012864,-0.067270,-0.835384,-0.888282,-0.936983,-1.112395,-0.905878,-0.510700,-0.747756,-0.887377,-1.037065,-0.827897,-0.926318


In [94]:
# Inverse transform for real values
pred_descaled_df = pd.DataFrame(all_scaler.inverse_transform(predictions_df), columns=predictions_df.columns, index=predictions_df.index)
pred_descaled_df

,Test Predictions,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2017-12-01,5.996657e+05,450.60,69.957500,43.761000,42.962000,1.25,131.8,143.8,139.2,121.4,94.5,136.9,125.0,114.3,163.1,129.5,126.5
2015-07-01,4.732359e+05,293.20,51.817727,36.800910,27.735454,0.75,127.1,140.3,133.8,119.8,94.7,128.7,120.7,109.6,152.1,124.5,121.4
2016-07-01,5.437448e+05,336.20,47.110500,35.170001,29.065500,0.75,128.5,142.4,136.0,122.2,94.2,128.2,122.1,111.5,156.9,125.9,123.8
2021-10-01,9.054109e+05,772.00,85.664286,74.742381,69.252381,0.50,143.9,160.4,155.4,126.2,94.0,155.7,133.8,119.7,176.3,140.8,136.6
2011-05-01,3.625625e+05,263.50,47.713809,18.571429,13.053333,1.25,120.0,127.5,125.2,110.2,92.3,126.6,116.9,105.4,135.7,118.4,114.7
2012-12-01,3.938521e+05,391.30,43.964500,26.084500,20.584000,1.25,122.0,131.7,127.5,113.6,92.0,127.7,118.8,106.4,138.3,120.1,116.5
2018-10-01,6.054083e+05,361.40,68.699565,35.084348,32.160435,2.00,134.2,146.3,142.0,123.8,94.7,140.3,126.3,116.1,169.8,131.8,128.5
2010-01-01,3.412174e+05,255.00,40.146316,15.553158,12.573684,0.50,115.8,122.2,121.8,108.1,92.8,117.6,114.1,103.5,131.1,114.5,112.1
2017-06-01,6.488483e+05,370.20,61.320455,38.350909,33.481819,0.75,129.9,143.0,137.8,122.4,93.3,129.8,124.2,114.1,160.8,127.3,125.3


In [95]:
total_chart = pd.concat([train_pred_descaled_df, pred_descaled_df],axis=1) 
total_chart['Actual'] = all_values['Composite'].values

In [96]:
total_chart[['Actual', 'Train Predictions', 'Test Predictions']].hvplot(
    xlabel='Date', 
    ylabel='Price', 
    title='Actual and Predicted Index',
    height=800,
    width=2000
).opts(
    yformatter=NumeralTickFormatter(format="0,0")
)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [97]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def weighted_absolute_percentage_error(y_true, y_pred):
    return 100 / len(y_true) * np.sum(np.abs((y_true - y_pred) / y_true))

def weighted_mean_absolute_percentage_error(y_true, y_pred):
    return 100 / sum(y_true) * np.sum(np.abs(y_true - y_pred))

In [98]:
# Generate predictions
y_pred = nn.predict(X_test)

# Calculate R-Squared
r2 = r2_score(y_test, y_pred)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Absolute Percentage Error
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

# Calculate Normalized Root Mean Squared Error
nrmse = rmse / (np.max(y_test) - np.min(y_test))

# Calculate Weighted Absolute Percentage Error
wape = weighted_absolute_percentage_error(y_test, np.squeeze(y_pred))

# Calculate Weighted Mean Absolute Percentage Error
wmape = weighted_mean_absolute_percentage_error(y_test, np.squeeze(y_pred))

2/2 [==============================] - 0s 3ms/step


In [99]:
print(f"R-Squared: {r2}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Normalized Root Mean Squared Error: {nrmse}")
print(f"Weighted Absolute Percentage Error: {wape}")
print(f"Weighted Mean Absolute Percentage Error: {wmape}")

R-Squared: 0.9973623638831222
Mean Absolute Error: 0.03132581635117842
Mean Absolute Percentage Error: 280.18593576661846
Mean Squared Error: 0.0021775098999228574
Root Mean Squared Error: 0.04666379645852722
Normalized Root Mean Squared Error: 0.012188115074936225
Weighted Absolute Percentage Error: 6.241233449984207
Weighted Mean Absolute Percentage Error: -20.490221404069977


In [100]:
nn_json = nn.to_json()

file_path = ("Models/model2.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

In [101]:
file_path = "Models/model2.h5"
nn.save_weights(file_path)

In [102]:
total_chart_save = pd.concat([train_pred_descaled_df['Train Predictions'], pred_descaled_df['Test Predictions']],axis=0) 
total_chart_save = total_chart_save.sort_index()
total_chart_save.rename("composite", inplace=True)
total_chart_save

date
2008-06-01    313345.350159
2008-07-01    317072.199913
2008-08-01    317819.902579
2008-09-01    318762.533613
2008-10-01    297809.840863
                  ...      
2022-12-01    875918.959025
2023-01-01    872817.286199
2023-02-01    866809.986384
2023-03-01    864310.931595
2023-04-01    877377.860393
Name: composite, Length: 179, dtype: float64

In [103]:
total_chart_save.to_csv('Resources/total_chart_save.csv', index=True)